In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# 2022年6月29日～

コストを算出するプログラムの作成


In [3]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]

conditionDict_toBuild: dict[str, list[int]] = {
    "processes": train_lulesh_processes,
    "iterations": train_lulesh_iterations,
    "sizes": train_lulesh_sizes,
}

conditionDict_InReal: dict[str, list[int]] = {
    "processes": test_lulesh_processes,
    "iterations": test_lulesh_iterations,
    "sizes": test_lulesh_sizes,
}

csvFilePath: str = csvDirPath + "ExecTime@LULESH.csv"
execTimeAtLulesh: pd.DataFrame = pd.read_csv(csvFilePath)

print(f"_toBuild")
computationTime_toBuild: float = 0
for index, row in execTimeAtLulesh.dropna().iterrows():

    if row["プロセス数"] not in conditionDict_toBuild["processes"]:
        continue
    if row["問題サイズ"] not in conditionDict_toBuild["sizes"]:
        continue
    if row["イテレーション数"] not in conditionDict_toBuild["iterations"]:
        continue
    computationTime_toBuild += row["実行時間（ｓ）"] * row["プロセス数"]

print(f"_InReal")
computationTime_InReal: float = 0
for index, row in execTimeAtLulesh.dropna().iterrows():

    if row["プロセス数"] not in conditionDict_InReal["processes"]:
        continue
    if row["問題サイズ"] not in conditionDict_InReal["sizes"]:
        continue
    if row["イテレーション数"] not in conditionDict_InReal["iterations"]:
        continue
    computationTime_InReal += row["実行時間（ｓ）"] * row["プロセス数"]

print(
    f"computationTime_toBuild={computationTime_toBuild}, computationTime_InReal={computationTime_InReal}"
)
print(f"相対コスト（％） = {computationTime_toBuild/computationTime_InReal * 100}")

_toBuild
_InReal
computationTime_toBuild=181695.41600000003, computationTime_InReal=614400.0
相対コスト（％） = 29.57282161458334


In [4]:
# NPB

benchmarkName = "cg"

train_NPB_processes: list[int] = [2, 4, 8, 16, 32, 64, 128]
train_NPB_classes: list[str] = ["A", "B", "C"]

test_NPB_processes: list[int] = [256]
test_NPB_classes: list[str] = ["D"]

conditionDict_toBuild: dict[str, list[int] | list[str]] = {
    "processes": train_NPB_processes,
    "classes": train_NPB_classes,
}

conditionDict_InReal: dict[str, list[int] | list[str]] = {
    "processes": test_NPB_processes,
    "classes": test_NPB_classes,
}

csvFilePath: str = csvDirPath + "ExecTime@" + benchmarkName + ".csv"
execTimeAtBenchmark: pd.DataFrame = pd.read_csv(csvFilePath)
execTimeAtBenchmark = execTimeAtBenchmark.set_index("Unnamed: 0")

print(execTimeAtBenchmark.index.is_unique)
print(execTimeAtBenchmark.columns.is_unique)
print(execTimeAtBenchmark.index)
print(execTimeAtBenchmark.columns)

print(f"_toBuild")
computationTime_toBuild: float = 0
for train_class in train_NPB_classes:
    for train_process in train_NPB_processes:
        execNum = execTimeAtBenchmark.at[str(train_class), str(train_process)]
        if math.isnan(execNum) == False:
            print(
                f"train_class={train_class},train_process={train_process},execNum={execNum}"
            )
            computationTime_toBuild += float(train_process) * float(execNum)

print(f"_inReal")
computationTime_InReal: float = 0
for test_class in test_NPB_classes:
    for test_process in test_NPB_processes:
        execNum = execTimeAtBenchmark.at[str(test_class), str(test_process)]
        if math.isnan(execNum) == False:
            print(
                f"test_class={test_class},test_process={test_process},execNum={execNum}"
            )
            computationTime_InReal += float(test_process) * float(execNum)

print(f"index = {execTimeAtBenchmark.index}\ncolumn = {execTimeAtBenchmark.columns}")
execTimeAtBenchmark

print(
    f"computationTime_toBuild={computationTime_toBuild}, computationTime_InReal={computationTime_InReal}"
)
print(f"相対コスト（％） = {computationTime_toBuild/computationTime_InReal * 100}")

True
True
Index(['S', 'W', 'A', 'B', 'C', 'D'], dtype='object', name='Unnamed: 0')
Index(['1', '2', '4', '8', '9', '16', '25', '32', '36', '49', '64', '81',
       '100', '121', '128', '144', '169', '196', '225', '256'],
      dtype='object')
_toBuild
train_class=A,train_process=2,execNum=0.43
train_class=A,train_process=4,execNum=0.22
train_class=A,train_process=8,execNum=0.11
train_class=A,train_process=16,execNum=0.08
train_class=A,train_process=32,execNum=0.07
train_class=A,train_process=64,execNum=0.05
train_class=A,train_process=128,execNum=0.05
train_class=B,train_process=2,execNum=47.78
train_class=B,train_process=4,execNum=25.64
train_class=B,train_process=8,execNum=7.38
train_class=B,train_process=16,execNum=4.08
train_class=B,train_process=32,execNum=3.13
train_class=B,train_process=64,execNum=1.5
train_class=B,train_process=128,execNum=0.89
train_class=C,train_process=2,execNum=155.79
train_class=C,train_process=4,execNum=82.68
train_class=C,train_process=8,execNum=35.51
tr

In [5]:
benchmarkName = "ep"

csvFilePath: str = csvDirPath + "ExecTime@" + benchmarkName + ".csv"
execTimeAtBenchmark: pd.DataFrame = pd.read_csv(csvFilePath)
execTimeAtBenchmark = execTimeAtBenchmark.set_index("Unnamed: 0")

print(execTimeAtBenchmark.index.is_unique)
print(execTimeAtBenchmark.columns.is_unique)
print(execTimeAtBenchmark.index)
print(execTimeAtBenchmark.columns)

print(f"_toBuild")
computationTime_toBuild: float = 0
for train_class in train_NPB_classes:
    for train_process in train_NPB_processes:
        execNum = execTimeAtBenchmark.at[str(train_class), str(train_process)]
        if math.isnan(execNum) == False:
            print(
                f"train_class={train_class},train_process={train_process},execNum={execNum}"
            )
            computationTime_toBuild += float(train_process) * float(execNum)

print(f"_inReal")
computationTime_InReal: float = 0
for test_class in test_NPB_classes:
    for test_process in test_NPB_processes:
        execNum = execTimeAtBenchmark.at[str(test_class), str(test_process)]
        if math.isnan(execNum) == False:
            print(
                f"test_class={test_class},test_process={test_process},execNum={execNum}"
            )
            computationTime_InReal += float(test_process) * float(execNum)

print(f"index = {execTimeAtBenchmark.index}\ncolumn = {execTimeAtBenchmark.columns}")
execTimeAtBenchmark

print(
    f"computationTime_toBuild={computationTime_toBuild}, computationTime_InReal={computationTime_InReal}"
)
print(f"相対コスト（％） = {computationTime_toBuild/computationTime_InReal * 100}")

True
True
Index(['S', 'W', 'A', 'B', 'C', 'D'], dtype='object', name='Unnamed: 0')
Index(['1', '2', '4', '8', '9', '16', '25', '32', '36', '49', '64', '81',
       '100', '121', '128', '144', '169', '196', '225', '256'],
      dtype='object')
_toBuild
train_class=A,train_process=2,execNum=6.48
train_class=A,train_process=4,execNum=3.27
train_class=A,train_process=8,execNum=1.62
train_class=A,train_process=16,execNum=0.81
train_class=A,train_process=32,execNum=0.44
train_class=A,train_process=64,execNum=0.42
train_class=A,train_process=128,execNum=0.13
train_class=B,train_process=2,execNum=25.92
train_class=B,train_process=4,execNum=13.17
train_class=B,train_process=8,execNum=6.51
train_class=B,train_process=16,execNum=3.26
train_class=B,train_process=32,execNum=1.7
train_class=B,train_process=64,execNum=0.85
train_class=B,train_process=128,execNum=0.46
train_class=C,train_process=2,execNum=103.67
train_class=C,train_process=4,execNum=51.91
train_class=C,train_process=8,execNum=26.05
tr

In [6]:
benchmarkName = "is"

csvFilePath: str = csvDirPath + "ExecTime@" + benchmarkName + ".csv"
execTimeAtBenchmark: pd.DataFrame = pd.read_csv(csvFilePath)
execTimeAtBenchmark = execTimeAtBenchmark.set_index("Unnamed: 0")

print(execTimeAtBenchmark.index.is_unique)
print(execTimeAtBenchmark.columns.is_unique)
print(execTimeAtBenchmark.index)
print(execTimeAtBenchmark.columns)

print(f"_toBuild")
computationTime_toBuild: float = 0
for train_class in train_NPB_classes:
    for train_process in train_NPB_processes:
        execNum = execTimeAtBenchmark.at[str(train_class), str(train_process)]
        if math.isnan(execNum) == False:
            print(
                f"train_class={train_class},train_process={train_process},execNum={execNum}"
            )
            computationTime_toBuild += float(train_process) * float(execNum)

print(f"_inReal")
computationTime_InReal: float = 0
for test_class in test_NPB_classes:
    for test_process in test_NPB_processes:
        execNum = execTimeAtBenchmark.at[str(test_class), str(test_process)]
        if math.isnan(execNum) == False:
            print(
                f"test_class={test_class},test_process={test_process},execNum={execNum}"
            )
            computationTime_InReal += float(test_process) * float(execNum)

print(f"index = {execTimeAtBenchmark.index}\ncolumn = {execTimeAtBenchmark.columns}")
execTimeAtBenchmark

print(
    f"computationTime_toBuild={computationTime_toBuild}, computationTime_InReal={computationTime_InReal}"
)
print(f"相対コスト（％） = {computationTime_toBuild/computationTime_InReal * 100}")

True
True
Index(['S', 'W', 'A', 'B', 'C', 'D'], dtype='object', name='Unnamed: 0')
Index(['1', '2', '4', '8', '9', '16', '25', '32', '36', '49', '64', '81',
       '100', '121', '128', '144', '169', '196', '225', '256'],
      dtype='object')
_toBuild
train_class=A,train_process=2,execNum=0.51
train_class=A,train_process=4,execNum=0.26
train_class=A,train_process=8,execNum=0.13
train_class=A,train_process=16,execNum=0.07
train_class=A,train_process=32,execNum=0.04
train_class=A,train_process=64,execNum=0.03
train_class=A,train_process=128,execNum=0.02
train_class=B,train_process=2,execNum=2.03
train_class=B,train_process=4,execNum=1.06
train_class=B,train_process=8,execNum=0.55
train_class=B,train_process=16,execNum=0.3
train_class=B,train_process=32,execNum=0.17
train_class=B,train_process=64,execNum=0.12
train_class=B,train_process=128,execNum=0.07
train_class=C,train_process=2,execNum=8.09
train_class=C,train_process=4,execNum=4.24
train_class=C,train_process=8,execNum=2.3
train_cla

In [7]:
benchmarkName = "mg"

csvFilePath: str = csvDirPath + "ExecTime@" + benchmarkName + ".csv"
execTimeAtBenchmark: pd.DataFrame = pd.read_csv(csvFilePath)
execTimeAtBenchmark = execTimeAtBenchmark.set_index("Unnamed: 0")

print(execTimeAtBenchmark.index.is_unique)
print(execTimeAtBenchmark.columns.is_unique)
print(execTimeAtBenchmark.index)
print(execTimeAtBenchmark.columns)

print(f"_toBuild")
computationTime_toBuild: float = 0
for train_class in train_NPB_classes:
    for train_process in train_NPB_processes:
        execNum = execTimeAtBenchmark.at[str(train_class), str(train_process)]
        if math.isnan(execNum) == False:
            print(
                f"train_class={train_class},train_process={train_process},execNum={execNum}"
            )
            computationTime_toBuild += float(train_process) * float(execNum)

print(f"_inReal")
computationTime_InReal: float = 0
for test_class in test_NPB_classes:
    for test_process in test_NPB_processes:
        execNum = execTimeAtBenchmark.at[str(test_class), str(test_process)]
        if math.isnan(execNum) == False:
            print(
                f"test_class={test_class},test_process={test_process},execNum={execNum}"
            )
            computationTime_InReal += float(test_process) * float(execNum)

print(f"index = {execTimeAtBenchmark.index}\ncolumn = {execTimeAtBenchmark.columns}")
execTimeAtBenchmark

print(
    f"computationTime_toBuild={computationTime_toBuild}, computationTime_InReal={computationTime_InReal}"
)
print(f"相対コスト（％） = {computationTime_toBuild/computationTime_InReal * 100}")

True
True
Index(['S', 'W', 'A', 'B', 'C', 'D'], dtype='object', name='Unnamed: 0')
Index(['1', '2', '4', '8', '9', '16', '25', '32', '36', '49', '64', '81',
       '100', '121', '128', '144', '169', '196', '225', '256'],
      dtype='object')
_toBuild
train_class=A,train_process=2,execNum=0.55
train_class=A,train_process=4,execNum=0.32
train_class=A,train_process=8,execNum=0.2
train_class=A,train_process=16,execNum=0.11
train_class=A,train_process=32,execNum=0.1
train_class=A,train_process=64,execNum=0.06
train_class=A,train_process=128,execNum=0.03
train_class=B,train_process=2,execNum=2.58
train_class=B,train_process=4,execNum=1.48
train_class=B,train_process=8,execNum=0.93
train_class=B,train_process=16,execNum=0.5
train_class=B,train_process=32,execNum=0.46
train_class=B,train_process=64,execNum=0.26
train_class=B,train_process=128,execNum=0.14
train_class=C,train_process=2,execNum=23.2
train_class=C,train_process=4,execNum=13.64
train_class=C,train_process=8,execNum=7.59
train_cla

# Extra-P に食わせる入力データの作成

参考：[File format documentation](https://github.com/extra-p/extrap/blob/master/docs/file-formats.md)

参考：[3パラメータの入力ファイルの例](https://github.com/extra-p/extrap/blob/master/tests/data/text/three_parameter_1.txt)

出力例（妄想）

```
PARAMETER a
PARAMETER b
PARAMETER c

POINTS ( a_1 b_1 c_1 ) ( a_2 b_2 c_2 ) ( a_3 b_3 c_3 )
...
POINTS ( a_n1 b_n1 c_n1 ) ( a_n2 b_n2 c_n2 ) ( a_n3 b_n3 c_n3 )

REGION reg
METRIC time

DATA ( a_1 b_1 c_1 )_1 ( a_1 b_1 c_1 )_2 ( a_1 b_1 c_1 )_3
DATA ( a_2 b_2 c_2 )_1 ( a_2 b_2 c_2 )_2 ( a_2 b_2 c_2 )_3
DATA ( a_3 b_3 c_3 )_1 ( a_3 b_3 c_3 )_2 ( a_3 b_3 c_3 )_3
...
DATA ( a_n1 b_n1 c_n1 )_1 ( a_n1 b_n1 c_n1 )_2 ( a_n1 b_n1 c_n1 )_3
DATA ( a_n2 b_n2 c_n2 )_1 ( a_n2 b_n2 c_n2 )_2 ( a_n2 b_n2 c_n2 )_3
DATA ( a_n3 b_n3 c_n3 )_1 ( a_n3 b_n3 c_n3 )_2 ( a_n3 b_n3 c_n3 )_3
```


In [8]:
csvFilePath: str = csvDirPath + "ExecTime@LULESH.csv"
execTimeAtLulesh: pd.DataFrame = pd.read_csv(csvFilePath)

execTimeAtLulesh

,プロセス数,問題サイズ,イテレーション数,実行時間（ｓ）,プロファイル収集済
0,1,16,8,NaN,NaN
1,1,16,16,NaN,NaN
2,1,16,32,NaN,NaN
3,1,16,64,NaN,NaN
4,1,16,128,NaN,NaN
...,...,...,...,...,...
319,512,24,16,0.58,1.0
320,512,24,32,1.20,1.0
321,512,24,64,2.30,1.0
322,512,24,128,4.60,1.0


In [9]:
list_ss_PARAMETER: list[str] = []
list_ss_POINTS: list[str] = []
list_ss_DATA: list[str] = []

# x : プロセス数
# y : 問題サイズ
# z : イテレーション数
list_ss_PARAMETER.append("PARAMETER x")
list_ss_PARAMETER.append("PARAMETER y")
list_ss_PARAMETER.append("PARAMETER z")

list_ss_POINTS.append("POINTS")

list_ss_DATA.append("REGION reg")
list_ss_DATA.append("METRIC time")

for index, row in execTimeAtLulesh.dropna().iterrows():
    # print(f"index={index}, row=\n{row}")

    list_ss_POINTS.append(f"( {row['プロセス数']} {row['問題サイズ']} {row['イテレーション数']} )")
    list_ss_DATA.append(f"DATA {row['実行時間（ｓ）']} {row['実行時間（ｓ）']} {row['実行時間（ｓ）']}")

str_ss_PARAMETER: str = "\n".join(list_ss_PARAMETER)
str_ss_PONITS: str = " ".join(list_ss_POINTS)
str_ss_DATA: str = "\n".join(list_ss_DATA)

filePathInputExtraP = "./extra-p_docker/share/input.txt"

with open(filePathInputExtraP, mode="w") as f:
    f.write(str_ss_PARAMETER)
    f.write("\n\n")
    f.write(str_ss_PONITS)
    f.write("\n\n")
    f.write(str_ss_DATA)

作成された入力データから下記のコマンドでモデルを算出した

`extrap --text ./extra-p_docker/share/input.txt`

$
T = -1.2943410415130208 + 0.2533461049298612 * z^{1} + 2.150869701234015e^{-07} * y^{3} * log_2 (y)^{1} * z^{1}
$



In [10]:
x = 512
y = 24
z = 256
execTime_p512s24i256 = (
    -1.2943410415130208
    + 0.2533461049298612 * z
    + 2.150869701234015 * 10**-7 * y**3 * math.log2(y) * z
)

execTime_p512s24i256

67.05224698068999

In [11]:
list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelLin"
    # "modelBasicTree",
]

In [12]:
# NPB_CG
train_npb_processes: list[int] = [2, 4, 8, 16, 32, 64, 128]
train_npb_sizes: list[str] = ["A", "B", "C"]

test_npb_processes: list[int] = [256]
test_npb_sizes: list[str] = ["D"]

weightedMAPEatCG: float = returnWeightedMapeScoreFromCondition(
    benchmarkName="cg",
    trainCondition={"processes": train_npb_processes, "sizes": train_npb_sizes},
    testCondition={"processes": test_npb_processes, "sizes": test_npb_sizes},
    list_modelName=list_modelName,
    csvDirPath=csvDirPath,
)

weightedMAPEatCG

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

174.8135170057732

In [13]:
# NPB_EP
train_npb_processes: list[int] = [2, 4, 8, 16, 32, 64, 128]
train_npb_sizes: list[str] = ["A", "B", "C"]

test_npb_processes: list[int] = [256]
test_npb_sizes: list[str] = ["D"]

weightedMAPEatEP: float = returnWeightedMapeScoreFromCondition(
    benchmarkName="ep",
    trainCondition={"processes": train_npb_processes, "sizes": train_npb_sizes},
    testCondition={"processes": test_npb_processes, "sizes": test_npb_sizes},
    list_modelName=list_modelName,
    csvDirPath=csvDirPath,
)

weightedMAPEatEP

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

0.0

In [14]:
# NPB_IS
train_npb_processes: list[int] = [2, 4, 8, 16, 32, 64, 128]
train_npb_sizes: list[str] = ["A", "B", "C"]

test_npb_processes: list[int] = [256]
test_npb_sizes: list[str] = ["D"]

weightedMAPEatIS: float = returnWeightedMapeScoreFromCondition(
    benchmarkName="is",
    trainCondition={"processes": train_npb_processes, "sizes": train_npb_sizes},
    testCondition={"processes": test_npb_processes, "sizes": test_npb_sizes},
    list_modelName=list_modelName,
    csvDirPath=csvDirPath,
)

weightedMAPEatIS

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

0.14096468916816277

In [15]:
# NPB_MG
train_npb_processes: list[int] = [2, 4, 8, 16, 32, 64, 128]
train_npb_sizes: list[str] = ["A", "B", "C"]

test_npb_processes: list[int] = [256]
test_npb_sizes: list[str] = ["D"]

weightedMAPEatMG: float = returnWeightedMapeScoreFromCondition(
    benchmarkName="mg",
    trainCondition={"processes": train_npb_processes, "sizes": train_npb_sizes},
    testCondition={"processes": test_npb_processes, "sizes": test_npb_sizes},
    list_modelName=list_modelName,
    csvDirPath=csvDirPath,
)

weightedMAPEatMG

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

7024.080516840863

***

---

___

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 


***

---

___